## Python for Computer Vision

### Description
This project demonstrates MNIST digit classification using a MLP neural network. The process includes Data preprocessing, hyperparameter tuning using RandomsSearch, model building using the cusom build_deep_nn function, and optimal model evaluation on a test set.

In [1]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import a1
from kerastuner.tuners import RandomSearch


2024-03-07 13:22:23.793807: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/5t/0bxhn3f90pv92yp3ffchl8pr0000gn/T/ipykernel_9757/433827705.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
#loading MNIST training and test datasets.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

### Data Preprocessing
* Here,the pixel values originally represented as integers are normalised scales between the range of 0-1 to help with training and testing the model. <br>
* Further, we have One-Hot Encoded the label values transforing thm from integer to binary vectors, where each vector has a length equal to the number of classes (10). It helps the nueral network in understanding the categorical nature of labels.

In [3]:

# Normalizing pixel values in the range [0, 1] for both training and test images.
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
train_labels = to_categorical(train_labels, num_classes = 10)
test_labels = to_categorical(test_labels,num_classes =10)

### Hyperparameter Search Space and Model Building Function
### build_model function
* The build_model function helps in defining a search space for hyperparameters using Keras Tuner's HyperParameters (hp) object.
* Hyperparameters include the number of hidden layers, the size of each hidden layer, and the dropout rate for regularization, the details for which are already provided.
* This function builds a neural network using a1.build_deep_nn with architecture parameters determined by the hyperparameters.
### Compiling
* This model is then compiled using Adam optimizer, categorical crossentropy loss and accuracy.
### Hyperparameter tuning using RandomSearch
* A RandomSearch tuner is initialized with the build_model function as the target, aiming to maximize validation accuracy.
* The search is conducted over a predefined number of trials (max_trials) with multiple executions per trial (executions_per_trial).
* The search is performed using the training data with a validation split, and the results are stored in the 'keras_tuner_logs' directory.
  

In [5]:
def build_model(hp):
    # Defining hyperparameters
    rows, columns, channels = 28, 28, 1  # MNIST image dimensions

    # Number of hidden layers
    num_hidden_layers = hp.Int('num_hidden_layers', 1, 3)

    # Size of hidden layers
    hidden_size = hp.Int('hidden_size', 32, 128, step=32)

    # Dropout rate of the final hidden layer
    dropout_rate = hp.Float('dropout_rate', 0, 0.5, step=0.1)

    # Output size and activation
    output_size = 10
    output_activation = 'softmax'

    # building neural network using build_deep_nn
    model = a1.build_deep_nn(rows, columns, channels, num_hidden_layers, [hidden_size] * num_hidden_layers,
                           [0] * (num_hidden_layers - 1) + [dropout_rate], output_size, output_activation)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model
    
# Initializing the RandomSearch tuner for hyperparamater optimization
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    executions_per_trial = 10,
    max_trials=5,  # You can adjust this value based on computational resources
    directory='keras_tuner_logs',
    project_name='mnist_tuning'
)
# Searching for the optimal hyperparameters using the training data with validation split
tuner.search(train_images, train_labels, epochs=5, validation_split=0.2)


Trial 5 Complete [00h 01m 19s]
val_accuracy: 0.9710833311080933

Best val_accuracy So Far: 0.9745583355426788
Total elapsed time: 00h 43m 46s


## Optimal Model
* The best hyperparameters are obtained from the tuner search are then extracted and a new model is built with them.
* The model is then trained for a specified number of epoch with a validation split

In [6]:
# extracting the best hyperparameters from the search
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# using the best hyperparamaters to build a model
optimal_model = build_model(best_hps)



In [7]:
#training the model on training data
optimal_model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 3s 1ms/step - loss: 0.2920 - accuracy: 0.9129 - val_loss: 0.1371 - val_accuracy: 0.9578
Epoch 2/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1249 - accuracy: 0.9618 - val_loss: 0.0992 - val_accuracy: 0.9708
Epoch 3/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0854 - accuracy: 0.9736 - val_loss: 0.0857 - val_accuracy: 0.9743
Epoch 4/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0675 - accuracy: 0.9796 - val_loss: 0.0839 - val_accuracy: 0.9748
Epoch 5/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0560 - accuracy: 0.9823 - val_loss: 0.0871 - val_accuracy: 0.9754
Epoch 6/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0456 - accuracy: 0.9850 - val_loss: 0.0951 - val_accuracy: 0.9732
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0343 - accuracy: 0.9889 - val_loss: 0.0891 - val_accuracy:

### Model Evaluation
* This trained model is then evaluated on a test dataset, and then test loss and accuracy are calculated.

In [8]:
# evaluating the model on the test dataset
test_loss, test_accuracy = optimal_model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_accuracy}")


313/313 [==============================] - 0s 642us/step - loss: 0.0983 - accuracy: 0.9770
Test Accuracy: 0.9769999980926514


### What are the hyperparameters of the optimal model?
Best Hyperparameters: <br>
Number of Hidden Layers: 2 <br>
Hidden Layer Size: 128 <br>
Dropout Rate: 0.2 <br>


### What are the accuracy results of the optimal model on the test set?
Test Accuracy: 0.9769999980926514 or 97.7%